In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel("Vietnam_Movies.xlsx")

In [3]:
df['Year_of_Release'] = df['Release Date']
df['Year_of_Release'] = pd.to_datetime(df['Year_of_Release'])
df['Year_of_Release'] = df['Year_of_Release'].dt.year
df['Year_of_Release'] = df['Year_of_Release'].fillna(0)
df['Year_of_Release'] = df['Year_of_Release'].astype(int)

In [4]:
count_by_year_and_genre = pd.DataFrame(columns=['release_year', 'genre', 'count'])

count_by_genre_year = {}

for index, row in df.iterrows():
    if isinstance(row['Genres'], str):
        genres = row['Genres']
        year = row['Year_of_Release']
        
        cleaned = genres.replace("[", "").replace("]", "").replace("'", "").replace('"', "")
        genre_list = cleaned.split(",")
        genre_list = [genre.strip() for genre in genre_list]
        
        for genre in genre_list:
            if genre not in count_by_genre_year:
                    count_by_genre_year[genre] = {}
                    
            if year not in count_by_genre_year[genre]:
                count_by_genre_year[genre][year] = 1
            else:
                count_by_genre_year[genre][year] += 1

count_data = []
for genre, year_count in count_by_genre_year.items():
    for year, count in year_count.items():
        count_data.append([genre, year, count])      

count_by_year_and_genre = pd.DataFrame(count_data, columns=['genre', 'release_year', 'count'])

In [5]:
count_by_year_and_genre_sorted = count_by_year_and_genre.groupby('genre', as_index=False).apply(lambda x: x.sort_values('release_year'))

count_data = []

for genre, group in count_by_year_and_genre_sorted.groupby('genre'):
    if genre != '':
        pre_count = 0
        pre_year = 1989

        for index, row in group.iterrows():
            year = row['release_year']
            count = row['count']

            for y in range(pre_year + 1, year):
                if (y >= 1990) and (y <= 2024):
                    count_data.append([genre, y, pre_count])
            
            count_data.append([genre, year, count + pre_count])
            pre_count = count + pre_count
            pre_year = year
            
        for y in range(pre_year + 1, 2025):
            if (y >= 1990) and (y <= 2024):
                count_data.append([genre, y, pre_count])
        
count_by_year_and_genre = pd.DataFrame(count_data, columns=['genre', 'year', 'count'])

C:\Users\HP\AppData\Local\Temp\ipykernel_19832\1431297738.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  count_by_year_and_genre_sorted = count_by_year_and_genre.groupby('genre', as_index=False).apply(lambda x: x.sort_values('release_year'))


In [6]:
count_by_year_and_genre.to_excel('summary2.xlsx', index=False)